In [13]:
# SecureString parameter stored in AWS System Manager that holds a GitHub personal access token.
parameter_name = 'github_token'

# Repositories for which reports will be generated.
repo_names = ['aws-amplify/amplify-android', 'aws-amplify/aws-sdk-android', 'awslabs/aws-mobile-appsync-sdk-android']

In [14]:
# Retrieve the GitHub token from SSM to prevent oops-I-pushed-credentials-to-GitHub uh-ohs.

import boto3
from github import Github

ssm = boto3.client('ssm')
response = ssm.get_parameter(Name=parameter_name, WithDecryption=True)
token = response['Parameter']['Value']
github = Github(token)

In [15]:
# Loop through each repository, grab all issues, and create a DataFrame for each.

import pandas as pd

repos = {}

for repo_name in repo_names:
    repo = github.get_repo(repo_name)
    issues = []

    for issue in repo.get_issues(state='all'):
        labels = pd.array([label.name for label in issue.labels])
        
        if issue.pull_request is None:
            issues.append([labels, issue.created_at, issue.closed_at])
        
    repos[repo_name] = pd.DataFrame(issues, columns=['labels', 'created_at', 'closed_at'])

In [16]:
# Using the DataFrames, process and grab the counts of issues. Go through each label and gets counts for each label and other applied labels.

from datetime import datetime
from IPython.display import display, HTML

seven_days_ago = datetime.now() - pd.Timedelta('7 days')
thirty_days_ago = datetime.now() - pd.Timedelta('30 days')

open_counts = []
label_counts = []

for repo_name in repo_names:
    r = repos[repo_name]
    labels = {label for labels in repos[repo_name].labels for label in labels}

    open_issues = r[~(r.closed_at > '1970-01-01')]
    last_week_open_issues = r[(r.created_at < seven_days_ago) & ~(r.closed_at < seven_days_ago)]
    last_month_open_issues = r[(r.created_at < thirty_days_ago) & ~(r.closed_at < thirty_days_ago)]
    open_issues_count = len(open_issues.index)
    last_week_open_issues_count = len(last_week_open_issues.index)
    last_month_open_issues_count = len(last_month_open_issues.index)
    
    open_counts.append([repo_name,
                        open_issues_count,
                        last_week_open_issues_count,
                        open_issues_count - last_week_open_issues_count,
                        last_month_open_issues_count,
                        open_issues_count - last_month_open_issues_count])
    
    for label in sorted(labels):
        label_mask = r.labels.apply(lambda l: label in l)
        issues = r[label_mask]
        label_open_issues = issues[~(issues.closed_at > '1970-01-01')]
        last_week_label_issues = issues[(issues.created_at < seven_days_ago) & ~(issues.closed_at < seven_days_ago)]
        last_month_label_issues = issues[(issues.created_at < thirty_days_ago) & ~(issues.closed_at < thirty_days_ago)]
        label_issues_count = len(label_open_issues.index)
        last_week_label_issues_count = len(last_week_label_issues.index)
        last_month_label_issues_count = len(last_month_label_issues.index)
        
        if label_issues_count or last_week_label_issues_count or last_month_label_issues_count:
            label_counts.append([repo_name,
                                 label,
                                 '',
                                 label_issues_count,
                                 last_week_label_issues_count,
                                 label_issues_count - last_week_label_issues_count,
                                 last_month_label_issues_count,
                                 label_issues_count - last_month_label_issues_count])

            for other_label in labels - {label}:
                other_label_mask = issues.labels.apply(lambda l: other_label in l)
                other_label_issues = issues[other_label_mask]
                other_lael_open_issues = other_label_issues[~(other_label_issues.closed_at > '1970-01-01')]


                if not other_label_issues.empty:
                    last_week_other_label_issues = other_label_issues[(other_label_issues.created_at < seven_days_ago) & ~(other_label_issues.closed_at < seven_days_ago)]
                    last_month_other_label_issues = other_label_issues[(other_label_issues.created_at < thirty_days_ago) & ~(other_label_issues.closed_at < thirty_days_ago)]
                    other_label_issues_count = len(other_lael_open_issues.index)
                    last_week_other_label_issues_count = len(last_week_other_label_issues.index)
                    last_month_other_label_issues_count = len(last_month_other_label_issues.index)

                    if other_label_issues_count or last_week_other_label_issues_count or last_month_other_label_issues_count:
                        label_counts.append([repo_name,
                                             label,
                                             other_label,
                                             other_label_issues_count,
                                             last_week_other_label_issues_count,
                                             other_label_issues_count - last_week_other_label_issues_count,
                                             last_month_other_label_issues_count,
                                             other_label_issues_count - last_month_other_label_issues_count])
                
open_df = pd.DataFrame(open_counts, columns=['Repo', 'Open', 'Last Week', 'WoW', 'Last Month', 'MoM'])
labels_df = pd.DataFrame(label_counts, columns=['Repo', 'Label', 'Other Label', 'Open', 'Last Week', 'WoW', 'Last Month', 'MoM'])

In [17]:
# Build the report for display.

pd.set_option('display.max_rows', 500)

display(HTML('<h1>Open Source – Ops Report</h1>'))
display(HTML(f'<em>Generated on {datetime.now()}'))

display(HTML('<h2>Open Issues</h2>'))
display(open_df)

display(HTML('<h2>Open Issues by Label</h2>'))

for repo_name in repo_names:
    display(HTML(f'<h3>{repo_name}</h3>'))
    display(labels_df[labels_df.Repo == repo_name])

,Repo,Open,Last Week,WoW,Last Month,MoM
0,aws-amplify/amplify-android,18,16,2,29,-11
1,aws-amplify/aws-sdk-android,119,124,-5,144,-25
2,awslabs/aws-mobile-appsync-sdk-android,43,43,0,44,-1


,Repo,Label,Other Label,Open,Last Week,WoW,Last Month,MoM
0,aws-amplify/amplify-android,API,,8,8,0,9,-1
1,aws-amplify/amplify-android,API,Improvement,1,1,0,1,0
2,aws-amplify/amplify-android,API,Bug,5,5,0,5,0
3,aws-amplify/amplify-android,API,Feature Request,2,2,0,3,-1
4,aws-amplify/amplify-android,Bug,,6,6,0,11,-5
5,aws-amplify/amplify-android,Bug,API,5,5,0,5,0
6,aws-amplify/amplify-android,Bug,DataStore,1,1,0,5,-4
7,aws-amplify/amplify-android,Bug,Clarification Needed,0,0,0,2,-2
8,aws-amplify/amplify-android,Bug,Core,0,0,0,1,-1
9,aws-amplify/amplify-android,Clarification Needed,,0,0,0,2,-2


,Repo,Label,Other Label,Open,Last Week,WoW,Last Month,MoM
40,aws-amplify/aws-sdk-android,APIGateway,,4,4,0,4,0
41,aws-amplify/aws-sdk-android,APIGateway,Feature Request,1,1,0,1,0
42,aws-amplify/aws-sdk-android,APIGateway,Usage Question,3,3,0,3,0
43,aws-amplify/aws-sdk-android,AWSMobileClient,,31,32,-1,40,-9
44,aws-amplify/aws-sdk-android,AWSMobileClient,IoT,1,1,0,1,0
45,aws-amplify/aws-sdk-android,AWSMobileClient,Documentation,2,2,0,2,0
46,aws-amplify/aws-sdk-android,AWSMobileClient,Bug,7,8,-1,10,-3
47,aws-amplify/aws-sdk-android,AWSMobileClient,SNS,0,0,0,1,-1
48,aws-amplify/aws-sdk-android,AWSMobileClient,DynamoDB,1,1,0,1,0
49,aws-amplify/aws-sdk-android,AWSMobileClient,Needs Info from Requester,2,2,0,6,-4


,Repo,Label,Other Label,Open,Last Week,WoW,Last Month,MoM
214,awslabs/aws-mobile-appsync-sdk-android,AppSync,,40,41,-1,42,-2
215,awslabs/aws-mobile-appsync-sdk-android,AppSync,Codegen,4,4,0,4,0
216,awslabs/aws-mobile-appsync-sdk-android,AppSync,Requesting Feedback,1,1,0,2,-1
217,awslabs/aws-mobile-appsync-sdk-android,AppSync,Question,9,9,0,10,-1
218,awslabs/aws-mobile-appsync-sdk-android,AppSync,closing-soon-if-no-response,0,1,-1,1,-1
219,awslabs/aws-mobile-appsync-sdk-android,AppSync,Documentation,2,2,0,2,0
220,awslabs/aws-mobile-appsync-sdk-android,AppSync,Infrastructure,2,2,0,2,0
221,awslabs/aws-mobile-appsync-sdk-android,AppSync,Product Review,2,2,0,2,0
222,awslabs/aws-mobile-appsync-sdk-android,AppSync,Pending,1,1,0,1,0
223,awslabs/aws-mobile-appsync-sdk-android,AppSync,Feature Request,12,13,-1,13,-1


In [18]:
seven_days_ago = datetime.now() - pd.Timedelta('7 days')

for repo_name in repo_names:
    r = repos[repo_name]
    labels = {label for labels in repos[repo_name].labels for label in labels}

    closed_issues = r[(r.closed_at > seven_days_ago)]
    display(repo_name, closed_issues.count())

'aws-amplify/amplify-android'

labels        0
created_at    0
closed_at     0
dtype: int64

'aws-amplify/aws-sdk-android'

labels        9
created_at    9
closed_at     9
dtype: int64

'awslabs/aws-mobile-appsync-sdk-android'

labels        1
created_at    1
closed_at     1
dtype: int64